<a href="https://colab.research.google.com/github/Pothuraju1690/pothuraju-2000/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Increasing the number of iterations can help to some extent when dealing with a small training or validation dataset, as it allows the model to see the data more times and potentially learn better. However, there are limits to how much improvement you can achieve with just more iterations. It's essential to strike a balance between the number of iterations and other hyperparameters like learning rate to prevent overfitting and achieve good generalization. Additionally, having a small dataset can still pose challenges, and it's often beneficial to explore techniques like data augmentation or transfer learning to make the most of limited data.
The size of the training and validation sets can have a significant impact on how well you can predict the accuracy on the test set using the validation set. Here's how it works:

1. *Larger Training Set*: If you have a larger training set, your model has more data to learn from. This can lead to a model that generalizes better to unseen data, including the test set. However, if the training set is too large and the validation set is too small, you might not detect overfitting issues because the model might perform very well on the small validation set even if it's overfitting.

2. *Smaller Validation Set*: A smaller validation set might lead to a less accurate estimate of your model's performance. It can be more sensitive to random variations, and the estimated accuracy may not be as reliable. This can make it harder to predict how well your model will perform on the test set accurately.

3. *Validation Set Size Balance*: Striking the right balance between the training and validation set sizes is crucial. You want your validation set to be large enough to provide a meaningful evaluation of your model's performance but not so large that it significantly reduces the size of your training set.

4. *Cross-Validation*: In cases where you have limited data, techniques like k-fold cross-validation can help. This involves splitting your data into k subsets and repeatedly training and validating your model on different combinations of these subsets. It can provide a more robust estimate of your model's performance.

5. *Randomness and Variability*: Keep in mind that machine learning experiments can have some randomness involved. Different random initializations, data shuffling, or even the specific data points in your validation set can lead to variability in validation results.

In summary, the size of the training and validation sets should be chosen carefully based on the amount of data available and your goals. It's important to strike a balance to ensure that your model generalizes well to unseen data while also obtaining reliable estimates of its performance. Cross-validation techniques can be valuable in scenarios where you have limited data or want a more robust estimate of your model's performance.

Balancing the size of your validation set depends on various factors, including the size of your dataset, the complexity of your model, and your specific goals. However, a common practice is to reserve around 10-20% of your dataset for validation. This allows you to have enough data to evaluate your model's performance while still having a substantial portion for training.

If you have a very large dataset, you might be able to get away with a smaller validation set (e.g., 5%), while for smaller datasets, you may want to allocate a larger percentage (e.g., 25%). Ultimately, the best percentage to reserve for validation should be determined through experimentation and cross-validation to ensure your model generalizes well to unseen data.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


The accuracy of the validation set can be affected when you change the percentage of data allocated to it during the training process.

1. *Increasing the Percentage of Validation Set:* When you increase the percentage of data allocated to the validation set, you are reducing the amount of data available for training. This can lead to a few outcomes:
   - *Pros:*
     - You might get a more accurate estimate of how well your model will perform on unseen data because the validation set is more representative of the test data.
   - *Cons:*
     - With less data for training, your model may not generalize well, and its performance on the training data may suffer.
     - Training might become slower as there's less data to learn from.

2. *Reducing the Percentage of Validation Set:* Conversely, if you reduce the percentage of dat
Yes, averaging validation accuracy across multiple splits, typically done in techniques like k-fold cross-validation, can provide more consistent and reliable results compared to a single validation split. This helps reduce the impact of randomness and provides a better estimate of a model's performance. It's a common practice in machine learning to assess the model's generalization ability.a
The accuracy of test estimates depends on the specific model or method being used. In machine learning, the accuracy of a test estimate typically refers to how well a model can predict unseen data. The accuracy of these estimates can be influenced by factors such as the quality and quantity of training data, the complexity of the model, and the evaluation metrics used.

If you have a specific model or method in mind, please provide more details, and I can give you more specific information.
The effect of the number of iterations on an estimate depends on the specific estimation method or algorithm being used. In many cases, increasing the number of iterations can lead to a more accurate estimate, but this is not always the case.

For iterative algorithms like numerical integration or optimization, increasing the number of iterations can generally lead to a better estimate because it allows the algorithm to refine its approximation and converge towards a more precise result. However, there may be diminishing returns, meaning that after a certain point, further iterations may not significantly improve the estimate and may increase computational cost.

In other cases, particularly in machine learning and statistical modeling, increasing the number of iterations in training algorithms can help improve model performance up to a point. This is often seen in training deep neural networks where more iterations (epochs) can lead to better model accuracy on a validation dataset, but there's a risk of overfitting if too many iterations are used.

So, the impact of the number of iterations on the estimate varies depending on the context and algorithm. It's essential to strike a balance between computational resources and the desired level of accuracy when deciding how many iterations to use.